In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('../data/external/potential-talents.csv').set_index('id')
data.shape

(104, 4)

In [3]:
data.drop_duplicates(inplace = True)
data.shape

(53, 4)

In [4]:
data.head()

,job_title,location,connection,fit
id,,,,
1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN
2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN
3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN


In [5]:
data.drop('fit', axis=1, inplace=True)

In [6]:
df_v1 = data.copy()
print(df_v1.shape)
df_v1.head()

(53, 3)


,job_title,location,connection
id,,,
1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85
2,Native English Teacher at EPIK (English Progra...,Kanada,500+
3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44
4,People Development Coordinator at Ryan,"Denton, Texas",500+
5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+


### Bag of words

In [7]:
import math
import re
from collections import Counter

In [8]:
WORD = re.compile(r'\w+')

def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])
    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)
    if not denominator:
        return 0.0
    else:
        return float(numerator/denominator)
    
def text_to_vector(text):
    return Counter(WORD.findall(text))

In [9]:
keywords = 'Aspiring Human Resources'

job_title_vectors = [text_to_vector(text) for text in df_v1['job_title']]
keyword_vectors = text_to_vector(keywords)

In [10]:
df_v1['bow_fit'] = [get_cosine(keyword_vectors, title_vector) for title_vector in job_title_vectors]
df_bow = df_v1.sort_values('bow_fit', ascending=False)
df_bow.head(10)

,job_title,location,connection,bow_fit
id,,,,
97,Aspiring Human Resources Professional,"Kokomo, Indiana Area",71,0.866025
3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,0.866025
6,Aspiring Human Resources Specialist,Greater New York City Area,1,0.866025
73,"Aspiring Human Resources Manager, seeking inte...","Houston, Texas Area",7,0.800641
74,Human Resources Professional,Greater Boston Area,16,0.666667
79,Liberal Arts Major. Aspiring Human Resources A...,"Baton Rouge, Louisiana Area",7,0.654654
100,Aspiring Human Resources Manager | Graduating ...,"Cape Girardeau, Missouri",103,0.629941
27,Aspiring Human Resources Management student se...,"Houston, Texas Area",500+,0.612372
72,Business Management Major and Aspiring Human R...,"Monroe, Louisiana Area",5,0.612372


### TF-IDF

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [12]:
df_v2 = data.copy()

In [13]:
documents = df_v2['job_title'].tolist()
keywords = 'Aspiring Human Resources'

In [14]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(documents)
keyword_vector = vectorizer.transform([keywords])
similarities = [cosine_similarity(keyword_vector, text_vector) for text_vector in tfidf_matrix]
similarities_list = []
for i in similarities:
    similarities_list.append(i.item())
df_v2['tfidf_fit'] = similarities_list
df_tfidf = df_v2.sort_values('tfidf_fit', ascending=False)
df_tfidf

,job_title,location,connection,tfidf_fit
id,,,,
97,Aspiring Human Resources Professional,"Kokomo, Indiana Area",71,0.761756
3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,0.761756
6,Aspiring Human Resources Specialist,Greater New York City Area,1,0.682828
73,"Aspiring Human Resources Manager, seeking inte...","Houston, Texas Area",7,0.576029
72,Business Management Major and Aspiring Human R...,"Monroe, Louisiana Area",5,0.415676
74,Human Resources Professional,Greater Boston Area,16,0.412834
27,Aspiring Human Resources Management student se...,"Houston, Texas Area",500+,0.398810
66,Experienced Retail Manager and aspiring Human ...,"Austin, Texas Area",57,0.391409
7,Student at Humber College and Aspiring Human R...,Kanada,61,0.370897


### GloVe

In [15]:
import torchtext

In [16]:
df_v3 = data.copy()
df_v3.shape

(53, 3)

In [17]:
## getting nan values for embedding when simple clearning is not done for a few job_titles
def simple_cleaning(text):
    text = re.sub(r'\s+', ' ', text).strip()
    return text.lower()
df_v3['job_title'] = df_v3['job_title'].apply(simple_cleaning)
df_v3.head()

,job_title,location,connection
id,,,
1,2019 c.t. bauer college of business graduate (...,"Houston, Texas",85
2,native english teacher at epik (english progra...,Kanada,500+
3,aspiring human resources professional,"Raleigh-Durham, North Carolina Area",44
4,people development coordinator at ryan,"Denton, Texas",500+
5,advisory board member at celal bayar university,"İzmir, Türkiye",500+


In [18]:
glove = torchtext.vocab.GloVe(name='6B', dim=100)

In [19]:
def string_to_glove_embedding(string):
    tokens = string.split()
    indices = [glove.stoi[token] for token in tokens if token in glove.stoi]
    vectors = glove.vectors[indices]
    vectors_array = vectors.numpy()
    embedding = vectors_array.mean(axis=0)
    return embedding

In [20]:
keywords = 'aspiring human resources'
embeddings = df_v3['job_title'].apply(string_to_glove_embedding)
text_embedding_list = [embedding for embedding in embeddings]
keyword_array = string_to_glove_embedding(keywords)

similarity_scores = [cosine_similarity(array.reshape(1, -1), keyword_array.reshape(1, -1))[0, 0] for array in text_embedding_list]

df_v3['gloVe_fit'] = similarity_scores
df_gloVe = df_v3.sort_values('gloVe_fit', ascending=False)
df_gloVe.head()

,job_title,location,connection,gloVe_fit
id,,,,
6,aspiring human resources specialist,Greater New York City Area,1,0.953001
97,aspiring human resources professional,"Kokomo, Indiana Area",71,0.948721
3,aspiring human resources professional,"Raleigh-Durham, North Carolina Area",44,0.948721
74,human resources professional,Greater Boston Area,16,0.922117
73,"aspiring human resources manager, seeking inte...","Houston, Texas Area",7,0.920849


### Word2Vec

In [21]:
df_v4 = data.copy()
df_v4.head()

,job_title,location,connection
id,,,
1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85
2,Native English Teacher at EPIK (English Progra...,Kanada,500+
3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44
4,People Development Coordinator at Ryan,"Denton, Texas",500+
5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+


In [22]:
def tokenized_corpus(text):
    tokens = text.split()
    return tokens

toks = df_v4['job_title'].apply(tokenized_corpus)
tokenized_list = toks.tolist()

In [23]:
from gensim.models import Word2Vec

In [24]:
model = Word2Vec(tokenized_list, vector_size=100, window=5, min_count=2, workers=4)
model.save('word2vec.model')

In [25]:
model = Word2Vec.load("word2vec.model")

In [26]:
def string_to_word2vec_embedding(string):
    tokens = string.split()
    vectors = [model.wv[token] for token in tokens if token in model.wv]
    embedding = np.mean(vectors, axis=0)
    return embedding

In [27]:
keywords = 'aspiring human resources'
embeddings_w2v = df_v4['job_title'].apply(string_to_word2vec_embedding)
text_embedding_list_w2v = [embedding for embedding in embeddings_w2v]
keyword_array_w2v = string_to_word2vec_embedding(keywords)

similarity_scores_w2v = [cosine_similarity(array.reshape(1, -1), keyword_array_w2v.reshape(1, -1))[0, 0] for array in text_embedding_list_w2v]

df_v4['w2v_fit'] = similarity_scores
df__w2v = df_v4.sort_values('w2v_fit', ascending=False)
df__w2v.head()

,job_title,location,connection,w2v_fit
id,,,,
6,Aspiring Human Resources Specialist,Greater New York City Area,1,0.953001
97,Aspiring Human Resources Professional,"Kokomo, Indiana Area",71,0.948721
3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,0.948721
74,Human Resources Professional,Greater Boston Area,16,0.922117
73,"Aspiring Human Resources Manager, seeking inte...","Houston, Texas Area",7,0.920849


### BERT

In [28]:
import transformers

In [29]:
model = transformers.BertModel.from_pretrained('bert-base-uncased')
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

In [30]:
def string_to_bert_embedding(string):
    input_ids = tokenizer.encode_plus(string, add_special_tokens=True, return_tensors='pt')
    output = model(input_ids)
    embedding = torch.mean(output.last_hidden_state, dim=1)
    return embedding

In [31]:
df_v5 = data.copy()

In [32]:
keywords = 'aspiring human resources'
embeddings_BERT = df_v5['job_title'].apply(lambda x: string_to_bert_embedding(str(x)))
embeddings_BERT


KeyError: 'Invalid key. Only three types of key are available: (1) string, (2) integers for backend Encoding, and (3) slices for data subsetting.'

In [33]:
model = transformers.AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
tokenizer = transformers.AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

C:\Users\KUDIPUDI\.conda\envs\dsEnv\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\KUDIPUDI\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [34]:
def string_to_sbert_embedding(string):
    input_ids = tokenizer.encode_plus(string, add_special_tokens=True, return_tensors='pt')
    output = model(input_ids)
    embedding = torch.mean(output.last_hidden_state, dim=1)
    return embedding

In [35]:
df_v6 = data.copy()

In [36]:
keywords = 'aspiring human resources'
embeddings_SBERT = df_v6['job_title'].apply(string_to_sbert_embedding)
embeddings_SBERT

TypeError: list indices must be integers or slices, not tuple